In [1]:
import tensorflow as tf
import numpy as np

from embedding import gen_word2vec as word2vec

In [2]:
sess = tf.InteractiveSession()

In [3]:
global_step = tf.Variable(0, name="global_step")
inc = global_step.assign_add(1)

init = tf.initialize_all_variables()
sess.run(init)

print inc.eval()
print inc.eval()
print inc.eval()

1
2
3


In [18]:
embed_dim = 200
num_neg_samples = 100

epochs_to_train = 15
learning_rate = 0.025
batch_size = 16
window_size = 5
min_count = 5
subsample = 1e-3

(words, counts, words_per_epoch,current_epoch, total_words_processed, x, y) = \
    word2vec.skipgram(
        filename = 'text8',
        batch_size = batch_size,
        window_size = window_size,
        min_count = min_count,
        subsample = subsample
    )

(vocab_words, vocab_counts, words_per_epoch) = sess.run([words, counts, words_per_epoch])

In [19]:
vocab_size = len(vocab_words)

id2word = vocab_words
word2id = {}

for i, w in enumerate(id2word):
    word2id[w] = i

print("Vocab size: ", vocab_size - 1, " + UNK")
print("Words per epoch: ", words_per_epoch)

('Vocab size: ', 71290, ' + UNK')
('Words per epoch: ', 17005207)


In [25]:
print x.eval()
print y.eval()
print "="*30

tmp = [[1,2,1],[2,3,1]]

print tmp
print tf.reduce_sum(tmp).eval()
print tf.reduce_sum(tmp, 0).eval()
print tf.reduce_sum(tmp, 1).eval()
print "="*30

a = np.ones([2,4])
b = np.ones([2,4])
print tf.matmul(a, b, transpose_b=True).eval()

[455 455 455 455 455 455 455 455  20  20  20  20  20  20  20  20]
[   7 1325  105  455   20 2733  363    7 3678    1   20   59  363    7   59
 2733]
[[1, 2, 1], [2, 3, 1]]
10
[3 5 2]
[4 6]
[[ 4.  4.]
 [ 4.  4.]]


In [7]:
# w_in : vocab_size x embed_dim
w_embed = tf.Variable(
    tf.random_uniform(
        [vocab_size, embed_dim],
        -0.5 / embed_dim,
        +0.5 / embed_dim
    ),
    name = "w_in"
)

# w_out : vocab_size x embed_dim
w = tf.Variable(tf.zeros([vocab_size, embed_dim]), name="w")
b = tf.Variable(tf.zeros([vocab_size]), name="b")

labels_matrix = tf.reshape(tf.cast(y, dtype=tf.int64), [batch_size, 1])

# Samples a set of classes using the provided (fixed) base distribution.
neg_sampled_ids, _, _ = (tf.nn.fixed_unigram_candidate_sampler(
    true_classes = labels_matrix,
    num_true = 1,
    num_sampled = num_neg_samples,
    unique = True,
    range_max = vocab_size,
    distortion = 0.75,
    unigrams = vocab_counts.tolist()))

# batch_size x embed_dim
x_emb = tf.nn.embedding_lookup(w_embed, x)

# batch_size x embed_dim
true_w = tf.nn.embedding_lookup(w, labels)

# batch_size x 1
true_b = tf.nn.embedding_lookup(b, labels)

# batch_size x embed_dim * batch_size_size x embed_dim = batch_size_size x embed_dim
# tf.reduce_sum(x, 1) : row끼리 더함
true_y_ = tf.reduce_sum(tf.mul(x_emb, true_w), 1) + true_b


########################
# For negative samples
########################

# num_neg_samples x embed_dim
neg_w = tf.nn.embedding_lookup(w, neg_sampled_ids)

# num_neg_samples x 1
neg_b = tf.nn.embedding_lookup(b, neg_sampled_ids)
neg_b = tf.reshape(neg_b, [num_neg_samples])

# [batch_size x num_neg_samples] + [num_neg_samples] <- 각 row 마다 neg_b 더함
neg_y_ = tf.matmul(x_emb, neg_w, transpose_b=True) + neg_b


###############################
# Negative Cross Entropy Loss
###############################

# Measures the probability error in discrete classification tasks in which each
# class is independent and not mutually exclusive. For instance, one could
# perform multilabel classification where a picture can contain both an elephant
# and a dog at the same time.
true_loss = tf.nn.sigmoid_cross_entropy_with_logits(true_y_, tf.ones_like(true_y_))

global_step = tf.Variable(0, name="global_step")

words_to_train = float(words_per_epoch * epochs_to_train)

lr = learning_rate * tf.maximum(
    0.001,
    1.0 - tf.cast(total_words_processed, tf.float32) / words_to_train)

inc = global_step.assign_add(1)

with tf.control_dependencies([inc]):
    train = 

SyntaxError: invalid syntax (<ipython-input-7-726cfa630067>, line 40)

In [27]:
tf.nn.sigmoid_cross_entropy_with_logits?

In [ ]:
word2vec.neg_train

In [ ]:
tf.nn.fixed_unigram_candidate_sampler?